# Segmenting and Clustering Neighborhoods in Toronto


## import the needed Library and read the zip code url and table, also install the library for the map, geo and kmean if not installed

In [2]:
## import the needed Library
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('package installed')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
package installed


## read the wikipedia data 

In [3]:

torontoWikiUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
torontoResponse = urllib.request.urlopen(torontoWikiUrl)
totontoData = torontoResponse.read().decode('utf-8')
zipCodeHtml= BeautifulSoup(totontoData, 'html.parser')
zipCodeTable = zipCodeHtml.find('table', class_='wikitable sortable')
zipCodeList=[]
for tr in zipCodeTable.find_all('tr'):
    trow = []
    for td in tr.find_all('td'):
        trow.append(td.text.strip('\n'))
    zipCodeList.append(trow)   

## construct the dataframe from html response
## The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
zipCodeDF = pd.DataFrame(zipCodeList, columns = ['PostalCode', 'Borough', 'Neighborhood'])
## remove the header html tag
zipCodeDF = zipCodeDF[1:]
zipCodeDF

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
zipCodeDF_borough = zipCodeDF[zipCodeDF.Borough != 'Not assigned']  
zipCodeDF_borough

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [6]:
zipCodeDF_borough.describe()

,PostalCode,Borough,Neighborhood
count,212,212,212
unique,103,11,210
top,M8Y,Etobicoke,Runnymede
freq,8,45,2


## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [19]:
zipCodeDF_borough.is_copy=False 
zipCodeDF_borough['Neighborhood'] = np.where(zipCodeDF_borough['Neighborhood'].values == 'Not assigned', 
                                                   zipCodeDF_borough['Borough'].values,
                                                   zipCodeDF_borough['Neighborhood'].values
                                            )
zipCodeDF_borough

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


## More than one neighborhood can exist in one postal code area.Rows will be combined into one row with the neighborhoods separated with a comma

In [20]:
#concat neighborhood
zipCodeDF_combined = zipCodeDF_borough.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
#convert into dataframe
zipCodeDF_combined = zipCodeDF_combined.to_frame().reset_index()
zipCodeDF_combined

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## print out the final DataFrame : using.shape() print the number of rows of your dataframe

In [21]:
zipCodeDF_combined.shape

(103, 3)

In [22]:
zipCodeDF_combined.shape[0]

103

## Get the geocode for each zip using file provide in the project

In [23]:
!wget -q -O 'canada_geo.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [24]:
geocode = pd.read_csv('canada_geo.csv')
geocode

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## Merge the lat and lon into the processing zip code data

In [26]:
zipCodeDF_geo = zipCodeDF_combined.set_index('PostalCode').join(geocode.set_index('Postal Code')).reset_index()
zipCodeDF_geo.shape
zipCodeDF_geo.columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
zipCodeDF_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Use geopy library to get the latitude and longitude values of toronto.

In [27]:
address = 'TORONTO, ON'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Create a map of Toronto with neighborhoods superimposed on top

In [28]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(zipCodeDF_geo['Latitude'], zipCodeDF_geo['Longitude'], zipCodeDF_geo['Borough'], zipCodeDF_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##    ## Define FourSquare credential and version

In [59]:
# The code was removed by Watson Studio for sharing.

## Using a neighbourhood to explore the foursquare API

In [30]:
## record 93 is a neighbor does not have venues information from foursquare 
neighborhood_latitude = zipCodeDF_geo.loc[93, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = zipCodeDF_geo.loc[93, 'Longitude'] # neighborhood longitude value

neighborhood_name = zipCodeDF_geo.loc[93, 'Neighborhood'] # neighborhood name

borough_name = zipCodeDF_geo.loc[93, 'Borough'] # Borough name

print('Latitude and longitude values of {} at {} are {}, {}.'.format(neighborhood_name,
                                                               borough_name,
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington Avenue at Etobicoke are 43.6678556, -79.53224240000002.


In [36]:
## record 101
neighborhood_latitude = zipCodeDF_geo.loc[101, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = zipCodeDF_geo.loc[101, 'Longitude'] # neighborhood longitude value

neighborhood_name = zipCodeDF_geo.loc[101, 'Neighborhood'] # neighborhood name

borough_name = zipCodeDF_geo.loc[101, 'Borough'] # Borough name

print('Latitude and longitude values of {} at {} are {}, {}.'.format(neighborhood_name,
                                                               borough_name,
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown at Etobicoke are 43.739416399999996, -79.5884369.


In [37]:

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3e66601ed2193f6b22cb5c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b04a05bf964a520c45522e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '4b04a05bf964a520c45522e3',
       'location': {'address': '1530 Albion Rd',
        'cc': 'CA',
        'city': 'Etobicoke',
        'country': 'Canada',
        'crossStreet': 'at Finch Ave. W.',
        'distance': 318,
        'formattedAddress': ['1530 Albion Rd (at Finch Ave. W.)',
         'Etobicoke ON M9V 1B4',
         'Canada'],
    

In [38]:
## get category as New York excerise
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sheriff's No Frills,Grocery Store,43.741968,-79.586639
1,Subway,Sandwich Place,43.742421,-79.589471
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Shoppers Drug Mart,Pharmacy,43.740832,-79.583347
4,The Beer Store,Beer Store,43.741694,-79.584373


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
## get one of the borough
Etobicoke_data =  zipCodeDF_geo[zipCodeDF_geo.Borough =='Etobicoke'].reset_index(drop=True)   
#Etobicoke_data =  zipCodeDF_geo[zipCodeDF_geo.Neighborhood =='Neighborhood'].reset_index(drop=True)   

Etobicoke_venues = getNearbyVenues(names=Etobicoke_data['Neighborhood'],
                                  latitudes=Etobicoke_data['Latitude'],
                                   longitudes=Etobicoke_data['Longitude']
                                )
Etobicoke_venues

Humber Bay Shores, Mimico South, New Toronto
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Northwest


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
3,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café
4,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,McDonald's,43.602470,-79.498963,Fast Food Restaurant
5,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Subway,43.602382,-79.498275,Sandwich Place
6,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Popeyes Louisiana Kitchen,43.602069,-79.499400,Fried Chicken Joint
7,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Shoppers Drug Mart,43.601611,-79.502164,Pharmacy
8,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Maple Leaf House,43.602040,-79.498678,American Restaurant
9,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Halibut House Fish and Chips Inc.,43.601960,-79.501147,Seafood Restaurant


In [43]:
print(Etobicoke_venues.shape)
Etobicoke_venues.head()

(71, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
3,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café
4,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,McDonald's,43.602470,-79.498963,Fast Food Restaurant


In [44]:
# check how many neighborhood beling processed
Etobicoke_venues.groupby('Neighborhood').count().shape

(11, 6)

In [45]:
print('There are {} uniques categories.'.format(len(Etobicoke_venues['Venue Category'].unique())))

There are 39 uniques categories.


In [46]:
# one hot encoding
Etobicoke_venues.head()
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]
Etobicoke_onehot.shape

(71, 40)

In [47]:
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
Etobicoke_grouped
#Etobicoke_data

,Neighborhood,American Restaurant,Bakery,Bank,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Social Club,Supplement Shop,Wings Joint
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.0,0.0,0.100000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.100000,0.000000,0.0,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.1,0.0,0.000000,0.000000,0.100000,0.000000,0.1,0.000000,0.000000,0.000000
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.00,0.166667,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Cloverdale, Islington, Martin Grove, Princess ...",0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Humber Bay Shores, Mimico South, New Toronto",0.066667,0.066667,0.0,0.0,0.000000,0.000000,0.00,0.133333,0.000000,...,0.0,0.0,0.066667,0.000000,0.066667,0.066667,0.0,0.000000,0.000000,0.000000
5,"Humber Bay, King's Mill Park, Kingsway Park So...",0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,"Kingsview Village, Martin Grove Gardens, Richv...",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.25,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0.090909,0.0,0.0,0.000000,0.090909,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.090909,0.000000,0.0,0.090909,0.090909,0.090909
8,Northwest,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [48]:
## get the top 5 venues for each neighborhood
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
Etobicoke_grouped
#Etobicoke_data

,Neighborhood,American Restaurant,Bakery,Bank,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Social Club,Supplement Shop,Wings Joint
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.0,0.0,0.100000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.100000,0.000000,0.0,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.1,0.0,0.000000,0.000000,0.100000,0.000000,0.1,0.000000,0.000000,0.000000
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.00,0.166667,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Cloverdale, Islington, Martin Grove, Princess ...",0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Humber Bay Shores, Mimico South, New Toronto",0.066667,0.066667,0.0,0.0,0.000000,0.000000,0.00,0.133333,0.000000,...,0.0,0.0,0.066667,0.000000,0.066667,0.066667,0.0,0.000000,0.000000,0.000000
5,"Humber Bay, King's Mill Park, Kingsway Park So...",0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,"Kingsview Village, Martin Grove Gardens, Richv...",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.25,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0.090909,0.0,0.0,0.000000,0.090909,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.090909,0.000000,0.0,0.090909,0.090909,0.090909
8,Northwest,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [49]:
## define a function to get the top N venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Etobicoke_grouped['Neighborhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(11, 11)

In [51]:
## use Kmean clustering to cluster the neigborhood
# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 3, 0, 1, 0, 0, 2, 4, 0], dtype=int32)

In [55]:
## append the cluster label with Etobicoke data
Etobicoke_merged = Etobicoke_data
# merge Etobicoke_grouped with Etobicoke_data to add latitude/longitude for each neighborhood
# one of the neighborhood does not have venues information, so using inner join will filter out those data
# another option is coded those no venues neighborhood into negative cluster

Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), how = 'inner', on='Neighborhood')
# add clustering labels
Etobicoke_merged['Cluster Labels'] = kmeans.labels_
# add clustering labels
Etobicoke_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Café,Gym,Pizza Place,Bakery,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Liquor Store,Mexican Restaurant,Pharmacy,0
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,Pizza Place,Pub,Dance Studio,Coffee Shop,Pharmacy,Pool,Gym,Sandwich Place,Skating Rink,Beer Store,0
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,Pool,River,Park,Chinese Restaurant,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store,0
3,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,Baseball Field,Wings Joint,Coffee Shop,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store,3
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,Wings Joint,Supplement Shop,Bakery,Burger Joint,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Gym,Social Club,0
6,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,Golf Course,Bank,Coffee Shop,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store,1
7,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201,Pharmacy,Liquor Store,Beer Store,Café,Convenience Store,Pizza Place,Wings Joint,Flower Shop,Fast Food Restaurant,Drugstore,0
8,M9P,Etobicoke,Westmount,43.696319,-79.532242,Pizza Place,Chinese Restaurant,Intersection,Sandwich Place,Middle Eastern Restaurant,Coffee Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,0
9,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,Pizza Place,Bus Line,Mobile Phone Shop,Park,Wings Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,2
10,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,Grocery Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Sandwich Place,Beer Store,Liquor Store,Fried Chicken Joint,Pharmacy,Drugstore,4


In [57]:
## display the labeled data in map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'], 
                                  Etobicoke_merged['Neighborhood'], Etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters